In [ ]:
Loretta Gray 7.3 Decision Tree Regression Commmeted Hw6

In [ ]:
'''
This cell initializes Spark and creates a SparkSession.
1. Import findspark to configure PySpark for local environment.
2. Call findspark.init() to initialize Spark settings.
3. Import SparkSession from pyspark.sql.
4. Create a SparkSession using a builder pattern to configure the application name and any necessary settings.
'''

import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Python Spark regression example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/02/11 02:51:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/02/11 02:51:13 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/02/11 02:51:13 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
25/02/11 02:51:13 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


In [ ]:
'''
This cell reads a CSV file into a DataFrame using Spark.
1. Use spark.read.format('csv') to specify the file format as CSV.
2. Set options for the CSV file:
   - header='true' to treat the first row as the header.
   - inferschema='true' to automatically infer column data types.
3. Load the CSV file from the specified path and store it in a DataFrame 'df'.
'''

df = spark.read.format('csv').\
                       options(header='true', \
                       inferschema='true').\
            load("file:///Users/ellegreyllc/Desktop/Advertising.csv",header=True)


In [ ]:
'''
This cell displays the first 5 rows of the DataFrame and prints its schema.
1. df.show(5, True) shows the first 5 rows of the DataFrame, including column names and data types.
2. df.printSchema() prints the schema of the DataFrame, showing column names and inferred data types.
'''

df.show(5,True)
df.printSchema()


+-----+-----+---------+-----+
|   TV|Radio|Newspaper|Sales|
+-----+-----+---------+-----+
|230.1| 37.8|     69.2| 22.1|
| 44.5| 39.3|     45.1| 10.4|
| 17.2| 45.9|     69.3|  9.3|
|151.5| 41.3|     58.5| 18.5|
|180.8| 10.8|     58.4| 12.9|
+-----+-----+---------+-----+
only showing top 5 rows

root
 |-- TV: double (nullable = true)
 |-- Radio: double (nullable = true)
 |-- Newspaper: double (nullable = true)
 |-- Sales: double (nullable = true)



In [ ]:
'''
This cell provides summary statistics for the DataFrame.
1. df.describe() generates summary statistics for all numeric columns (e.g., count, mean, standard deviation, min, max).
2. .show() displays the summary statistics in the output.
'''

df.describe().show()


25/02/11 02:59:38 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+-----------------+------------------+------------------+------------------+
|summary|               TV|             Radio|         Newspaper|             Sales|
+-------+-----------------+------------------+------------------+------------------+
|  count|              200|               200|               200|               200|
|   mean|         147.0425|23.264000000000024|30.553999999999995|14.022500000000003|
| stddev|85.85423631490805|14.846809176168728| 21.77862083852283| 5.217456565710477|
|    min|              0.7|               0.0|               0.3|               1.6|
|    max|            296.4|              49.6|             114.0|              27.0|
+-------+-----------------+------------------+------------------+------------------+



In [ ]:
#Convert the data to dense vector (features and label)

In [ ]:
'''
This cell transforms the DataFrame into a format suitable for machine learning.
1. Import Row from pyspark.sql and Vectors from pyspark.ml.linalg.
2. Convert the DataFrame to an RDD and use map() to transform each row:
   - r[-1] extracts the last column as the label.
   - Vectors.dense(r[:-1]) converts the remaining columns into a dense feature vector.
3. Convert the transformed RDD back to a DataFrame with columns 'label' and 'features'.
4. Display the first 5 rows of the transformed DataFrame.
'''

from pyspark.sql import Row
from pyspark.ml.linalg import Vectors

transformed = df.rdd.map(lambda r: [r[-1], Vectors.dense(r[:-1])]).toDF(['label','features'])
transformed.show(5)



+-----+-----------------+
|label|         features|
+-----+-----------------+
| 22.1|[230.1,37.8,69.2]|
| 10.4| [44.5,39.3,45.1]|
|  9.3| [17.2,45.9,69.3]|
| 18.5|[151.5,41.3,58.5]|
| 12.9|[180.8,10.8,58.4]|
+-----+-----------------+
only showing top 5 rows



In [ ]:
#You will find out that all of the machine learning algorithms in Spark are based on
#the features and label. That is to say, you can play with all of the machine learning
#algorithms in Spark when you get ready the features and label.

#Deal with categorical variables

In [ ]:
'''
This cell sets up a feature indexer for machine learning and applies it to the DataFrame.
1. Import necessary modules for pipeline, regression, and evaluation.
2. VectorIndexer automatically identifies categorical features and indexes them:
   - inputCol specifies the input column with features.
   - outputCol specifies the output column where the indexed features will be stored.
   - maxCategories=4 ensures that features with more than 4 distinct values are treated as continuous.
3. Fit the feature indexer to the transformed data and transform it to create indexed features.
'''

from pyspark.ml import Pipeline
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator

# Automatically identify categorical features, and index them.
# We specify maxCategories so features with > 4
# distinct values are treated as continuous.

featureIndexer = VectorIndexer(inputCol="features", \
                               outputCol="indexedFeatures",\
                               maxCategories=4).fit(transformed)

data = featureIndexer.transform(transformed)


In [ ]:
'''
This cell displays the first 5 rows of the transformed DataFrame with indexed features.
1. data.show(5) shows the first 5 rows of the DataFrame with the indexed features, allowing you to check how the transformation has been applied.
'''

data.show(5)


+-----+-----------------+-----------------+
|label|         features|  indexedFeatures|
+-----+-----------------+-----------------+
| 22.1|[230.1,37.8,69.2]|[230.1,37.8,69.2]|
| 10.4| [44.5,39.3,45.1]| [44.5,39.3,45.1]|
|  9.3| [17.2,45.9,69.3]| [17.2,45.9,69.3]|
| 18.5|[151.5,41.3,58.5]|[151.5,41.3,58.5]|
| 12.9|[180.8,10.8,58.4]|[180.8,10.8,58.4]|
+-----+-----------------+-----------------+
only showing top 5 rows



In [ ]:
#Fit Decision Tree Regression Model

In [ ]:
'''
This cell sets up a DecisionTreeRegressor for training the model.
1. Import DecisionTreeRegressor from pyspark.ml.regression.
2. Instantiate a DecisionTreeRegressor model, specifying the input column for features (indexedFeatures) that will be used for training.
'''

from pyspark.ml.regression import DecisionTreeRegressor

# Train a DecisionTree model.
dt = DecisionTreeRegressor(featuresCol="indexedFeatures")


In [ ]:
'''
This cell splits the data into training and test datasets.
1. Use data.randomSplit() to randomly split the data into training (80%) and test (20%) datasets.
2. The seed ensures the split is reproducible.
3. Show the first 5 rows of the training dataset to verify the split.
'''

# split data into training and test datasets
trainingData, testData = data.randomSplit([0.8, 0.2], seed=1234)
trainingData.show(5)



+-----+----------------+----------------+
|label|        features| indexedFeatures|
+-----+----------------+----------------+
|  1.6|  [0.7,39.6,8.7]|  [0.7,39.6,8.7]|
|  4.8|   [8.6,2.1,1.0]|   [8.6,2.1,1.0]|
|  5.3|  [5.4,29.9,9.4]|  [5.4,29.9,9.4]|
|  5.5| [7.3,28.1,41.4]| [7.3,28.1,41.4]|
|  5.6|[13.2,15.9,49.6]|[13.2,15.9,49.6]|
+-----+----------------+----------------+
only showing top 5 rows



In [ ]:
#Pipeline Architecture

In [ ]:
'''
This cell chains the feature indexer and decision tree into a Pipeline and trains the model.
1. Create a Pipeline by specifying the stages:
   - featureIndexer: the feature indexing step.
   - dt: the decision tree regression model.
2. Fit the pipeline to the training data to create the model.
'''

# Chain indexer and decision tree in a Pipeline
pipeline = Pipeline(stages=[featureIndexer, dt])

model = pipeline.fit(trainingData)


In [ ]:
#Make predictions

In [ ]:
'''
This cell makes predictions using the trained model on the test data.
1. Use model.transform(testData) to apply the trained pipeline model to the test dataset.
2. This will generate predictions, which include the predicted label and features.
'''

# Make predictions.
predictions = model.transform(testData)


In [ ]:
'''
This cell selects and displays specific columns from the predictions.
1. Use predictions.select() to select the "features", "label", and "prediction" columns.
2. .show(5) displays the first 5 rows of the selected columns to review the predictions.
'''

# Select example rows to display.
predictions.select("features","label","prediction").show(5)


+----------------+-----+-----------------+
|        features|label|       prediction|
+----------------+-----+-----------------+
|  [4.1,11.6,5.7]|  3.2|              5.7|
| [13.1,0.4,25.6]|  5.3|5.466666666666666|
| [53.5,2.0,21.4]|  8.1|              8.4|
| [66.1,5.8,24.2]|  8.6|             10.1|
|[16.9,43.7,89.4]|  8.7|             8.96|
+----------------+-----+-----------------+
only showing top 5 rows



In [ ]:
#Evaluation

In [ ]:
'''
This cell evaluates the model's performance using the Root Mean Squared Error (RMSE).
1. Import RegressionEvaluator from pyspark.ml.evaluation.
2. Instantiate a RegressionEvaluator to compute the RMSE:
   - labelCol specifies the column containing the true labels.
   - predictionCol specifies the column containing the predicted labels.
   - metricName="rmse" tells the evaluator to compute RMSE.
3. Use evaluator.evaluate(predictions) to calculate the RMSE based on the predictions.
'''

# from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.evaluation import RegressionEvaluator

# Select (prediction, true label) and compute test error
evaluator = RegressionEvaluator(labelCol="label",
                                predictionCol="prediction",
                                metricName="rmse")

rmse = evaluator.evaluate(predictions)


In [ ]:
'''
This cell calculates the R-squared (R²) score to evaluate the model's performance.
1. Convert the "label" and "prediction" columns of the predictions DataFrame into pandas DataFrames (y_true and y_pred).
2. Import sklearn.metrics to use the r2_score function.
3. Use sklearn.metrics.r2_score() to compute the R² score based on true labels (y_true) and predicted values (y_pred).
4. Print the R² score.
'''

y_true = predictions.select("label").toPandas()
y_pred = predictions.select("prediction").toPandas()

import sklearn.metrics
r2_score = sklearn.metrics.r2_score(y_true, y_pred)
print('r2_score: {0}'.format(r2_score))


r2_score: 0.9454161819062926


In [ ]:
'''
This cell extracts the feature importances from the trained model.
1. Access the last stage of the pipeline (the decision tree model) using model.stages[-1].
2. Use the .featureImportances attribute to retrieve the importance of each feature used by the model.
'''

model.stages[-1].featureImportances


SparseVector(3, {0: 0.6584, 1: 0.3329, 2: 0.0087})